# 9.1. О чем этот модуль
В этом модуле мы рассмотрим классические методы рекомендательных систем:

- Ассоциативные правила.
- Коллаборативная фильтрация.
- Алгоритмы SVD и ALS.

# 9.2. Характеристики рекомендательных систем
## Задача рекомендательной системы

Проинформировать пользователя о продукте, который ему может быть наиболее интересен в данный момент времени.

## КОМУ НУЖНЫ РЕКОМЕНДАТЕЛЬНЫЕ СИСТЕМЫ?

→ клиенту — получает информацию о наиболее подходящих для него товарах, интересных мультимедийных продуктах и т. д. 

→ сервису — зарабатывает на предоставлении услуг.

## СТЕПЕНЬ ПЕРСОНАЛИЗАЦИИ

→ Неперсональные рекомендации — когда вам рекомендуют то же самое, что всем остальным. 

→ Персональные рекомендации используют всю доступную информацию о клиенте.

→ Более продвинутый вариант — рекомендации на данных из текущей сессии. Вы посмотрели несколько товаров, и внизу страницы вам предлагаются похожие.

# 9.3. Ассоциативные правила

Для того чтобы сравнить ассоциативные правила по их силе (значимости) необходимо ввести несколько метрик — Support, Confidence и Lift.

In [53]:
import numpy as np
import pandas as pd

In [54]:
a = np.array([[1,1,1,1],[2, 0, 0, 0],[3,1,1,0],[4,1,1,1],[5,0,1,1]])
df = pd.DataFrame(a, columns = ['tr', 'cola', 'beer', 'diapers'])
df

,tr,cola,beer,diapers
0,1,1,1,1
1,2,0,0,0
2,3,1,1,0
3,4,1,1,1
4,5,0,1,1


## SUPPORT
$$sup(x) = \frac{ \{ t \in T; X \in t \} }{|T|}$$

Здесь  — это X itemset, в котором находится items, T — количество транзакций.

Можно охарактеризовать этот показатель как индекс частоты встречаемости конкретного продукта в имеющихся транзакциях. Это для того случая, если нам интересен один конкретный продукт (item).

Чаще нам бывает важно, насколько часто какие-то два продукта встречаются вместе. Для такого случая мы будем рассчитывать следующий вариант показателя:

$$sup(x_1 \cup x_2) = \frac{ \sigma (x_1 \cup x_2) }{|T|}$$

Предположим, что  есть несколько транзакций, в которых присутствует пиво, подгузники и кола. Мы считаем количество совместных транзакций, в которых есть пиво и подгузники, и делим на общее количество транзакций.

        Получается, Support такого правила равен 3/5, или 60 %. 

In [55]:
df[(df.beer == 1)&(df.diapers == 1)][['beer', 'diapers']]

,beer,diapers
0,1,1
3,1,1
4,1,1


In [56]:
supp_b_and_d = df[(df.beer == 1)&(df.diapers == 1)].shape[0]/df.shape[0]
supp_b_and_d

0.6

## CONFIDENCE

Этот показатель высчитывается на основе метрики Support.

$$conf(x_1 \cup x_2) = \frac{ supp (x_1 \cup x_2) }{sup(T)}$$

Он определяет, как часто правило срабатывает для всего датасета.

У нас есть Support пива и подгузников, которое мы посчитали. Также мы можем посчитать Support только пива. 
        
        Отношение этих двух Support будет равно 3/4, или 75 %.

In [57]:
supp_b = df[(df.beer == 1)].shape[0]/df.shape[0]
supp_b

0.8

In [58]:
np.around(supp_b_and_d/supp_b, 2)

0.75

## LIFT 

Ещё одна метрика — Lift. Она вычисляется следующим образом:

- вычисляется Support совместной встречаемости двух продуктов;
- делится на произведение Support каждого из этих продуктов.

$$ lift(x_1 \cup x_2) = \frac{ supp (x_1 \cup x_2) }{supp(x_1)*supp(x_2)} $$


$$ lift = \frac{Confidence}{Expectedconfidence} = \frac{Подгузники|Пиво}{Подгузники} $$

In [59]:
supp_d = df[(df.diapers == 1)].shape[0]/df.shape[0]
supp_d

0.6

In [60]:
np.around(supp_b_and_d/(supp_d*supp_b), 2)

1.25

## АЛГОРИТМ APRIORI

Рассмотрим один из алгоритмов для построения ассоциативных правил — Apriori.

Apriori использует следующее утверждение: 

если X⊆Y, то supp(X) ≥ supp(Y).

Отсюда следуют два свойства:

- если Y встречается часто, то любое подмножество X: X⊆Y также встречается часто
- если X встречается редко, то любое супермножество Y: Y ⊇X также встречается редко

Apriori по уровням проходит по префиксному дереву и рассчитывает частоту X встречаемости подмножеств  в D. 

Таким образом:

- исключаются редкие подмножества и все их супермножества.
- рассчитывается supp(X) для каждого подходящего кандидата X размера k на уровне k.

## Задание 9.3.1

1. Какой показатель определяет частоту срабатываемости правила на нашем датасете?
- support
- confidence верно
- lift

2. Предположим, у нас есть данные о 300 транзакциях. Мы отметили, что в 62 из них покупают вместе форель и соевый соус. Рассчитайте показатель support, округлите до сотых (пример: 0.99).
- 0.21  верно 

3. Какие данные наиболее удобны для работы с ассоциативными правилами?
- Порядковые
- Бинарные верно
- Количественные
- Номинальные

## 9.4. Практика

In [1]:
import pandas as pd
df = pd.read_csv('data_fin.csv', sep=';')

Здесь есть ID для пользователя, ID для фильма и рейтинг, который поставил данный пользователь данному фильму.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_Id   int64  
 1   Rating    float64
 2   Movie_Id  int64  
dtypes: float64(1), int64(2)
memory usage: 2.2 GB


Из полученного датасета возьмем только те записи, у которых наивысший рейтинг (5) и объединим их по "Cust_Id". Фильмы сгруппируем в строчку с разделителем "пробел" так, чтобы для каждого пользователя была строка с Id тех фильмов, которые ему понравились:

In [63]:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [64]:
good.head()

Cust_Id
6     175 457 886 1467 2372 2452 2782 3290 4043 4633...
7     8 30 83 175 257 283 285 313 357 457 458 468 50...
8     1202 1799 1905 2186 3610 3925 4306 5054 5317 5...
10    473 985 1542 1905 2172 3124 3371 3962 4043 430...
25                4432 6786 7605 9326 10643 15107 15270
Name: Movie_Id, dtype: object

Сначала идёт ID пользователя, дальше через пробел фильмы, которые нравятся этому пользователю.

In [65]:
import apyori

Cделаем несколько ассоциативных правил. Мы можем регулировать их количество, меняя параметры алгоритмов. Посмотрим, какие ассоциативные правила получаются для support = 0.1.

In [66]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)

Строго говоря, мы получили не сами ассоциативные правила, а генератор. Это можно проверить, если, например, вызвать переменную association_rules:

In [67]:
association_rules

<generator object apriori at 0x000001B7D747FDC8>

In [68]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [69]:
asr_df.sample(10)

,from,to,confidence,support,lift
1718,11521,2452 5582 14961 14240,0.259398,0.0540124,4.59819
583,10042,11283 2782,0.260829,0.040732,3.1777
466,17157,7230,0.314779,0.0447288,2.53424
611,10042,2452 14240,0.470967,0.0735479,3.03236
79,11064,6974,0.414779,0.0627631,3.23419
534,3938,3962,0.511215,0.0716239,2.98322
1869,14240,7057 2782 2452 7230,0.206185,0.0412734,4.77095
1178,2782,5582 9628,0.285592,0.0438315,2.90674
607,10042,14961 14240,0.368508,0.0575476,3.4082
1260,10042,16954 2452 7230,0.266298,0.0415861,5.2994


In [70]:
len(asr_df)

2023

Мы видим здесь таблицу, где 2023 ассоциативных правила, и для каждого рассчитаны известные нам показатели.

Для того чтобы перейти от Id фильмов, к их названиям, нужно загрузить еще один файл, в котором содержится Id фильма, год его производства и название:

In [71]:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

Мы можем написать процедуру, которая будет выводить названия фильмов в ассоциативном правиле и фильм, которое это ассоциативное правило рекомендует:

In [72]:
def get_rule_title(rule):
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id == int(rule['to'])]['Name'].values)

Можем посмотреть, как выглядит это правило. Например, вызовем сотое правило:

In [73]:
get_rule_title(asr_df.loc[99])

['Monsters']
----------->
['Finding Nemo (Widescreen)']


То есть, если человеку нравится фильм «Монстры», то мы советуем ему посмотреть фильм «В поисках Немо».

Перейдём к построению рекомендаций для случайного человека под id=159992. Посмотрим, какие фильмы он смотрел и как он их оценил. 

In [74]:
j = 159992
titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']

1144     The Wedding Planner
2151         What Women Want
5316       Miss Congeniality
6286            Pretty Woman
6385              Sister Act
7233            Men of Honor
10358          Runaway Bride
11148      Maid in Manhattan
13049       Two Weeks Notice
15581     Sweet Home Alabama
Name: Name, dtype: object

Как мы можем посчитать посчитать рекомендации для этого человека? Мы можем пройтись по всем правилам в нашей таблице и проверить, если они присутствуют в просмотрах человека и у них высокий рейтинг, значит это правило ему подходит и мы можем добавить этот фильм в список рекомендаций.

In [75]:
def print_rule_title(rule):
    return (titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
    

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

{'Pretty Woman', 'Dirty Dancing'}


Сложность такой рекомендательной системы состоит в том, что мы ограничены теми ассоциативными правилами, которые были созданы. Если фильм редкий, то он в эти ассоциативные правила, скорее всего, не попадёт.

Также для разных людей будет разное количество рекомендаций, в зависимости от того, сколько фильмов посмотрел каждый из них. И, если нам захочется порекомендовать каждому пользователю минимум три фильма, то не с каждым пользователем мы сможем это сделать.

Эту проблему можно решить с помощью алгоритма Коллаборативная фильтрация. 

## Задание 9.4.1

1. Найдите фильмы, которые понравились пользователю с ID, равным 130. Понравившимися пользователю фильмами мы будем считать те фильмы, которым он поставил наивысшую оценку (5). Скопируйте все ID фильмов. Например: 68 943 325 1234.

In [76]:
good[good.index == 130].values

array(['1865 3456 3962 5515 6029 6428 8159 8327 8782 8784 11165 11242 11701 12084 12232 14482'],
      dtype=object)

In [77]:
get_rule_title(asr_df.loc[315])

['The Patriot']
----------->
['The Green Mile']


In [78]:
j = 21
result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
result_df = pd.DataFrame(result, columns = ['film'])
result_df['len'] = result_df.film.apply(lambda x: len(x))

In [79]:
result_df[result_df.len == result_df.len.min()]['film'].iloc[0]

'The Sixth Sense'

# 9.5. Коллаборативная фильтрация

## В понимании этого материала мне помогли следующие ресурсы:

- [Коллаборативная фильтрация - К.В. Воронцов](https://youtu.be/kfhqzkcfMqI 'youtube')

## МАТРИЦА ПРЕДПОЧТЕНИЙ

Расположим в матрице клиентов по строкам, а продукты — по столбцам. На пересечении строк и столбцов разметим оценку клиентов на эти продукты.  То есть первый клиент поставил второму товару 3, а третий клиент поставил первому товару 2 и так далее

## КЛАСТЕРИЗАЦИЯ ПОЛЬЗОВАТЕЛЕЙ

Далее мы можем применить алгоритм кластеризации для того, чтобы объединить людей.
Выберем условную меру схожести пользователей по их истории оценок:

$$sim(u, \upsilon)$$

Объединим пользователей в группы (кластеры) 

Разбиваем их на кластеры так, чтобы похожие пользователи оказались в одном кластере, а непохожие — в разных.

Оценку пользователя объекту будем предсказывать как среднюю оценку кластера этому объекту.

Итак, у нас есть пользователь, и у нас спрашивают, как он оценил данный фильм. Мы смотрим на кластер пользователя. Смотрим оценки пользователей из этого кластера на этот фильм (оценки тех пользователей, которые смотрели этот фильм и поставили оценку). Берем их и усредняем. Это и есть предсказание оценки фильма для нашего пользователя.

### Проблемы алгоритма:

- Нечего рекомендовать новым/нетипичным пользователям. Если у нас появляется пользователь, который не похож ни на кого, то мы не знаем, в какой кластер его отнести. Конечно, мы первоначально относим его в какой-то случайный кластер, но рекомендации в таком случае будут плохие.

- Не учитывается специфика каждого пользователя. По сути, мы выявляем некоторые шаблонные паттерны поведения и предпочтений, и для каждого паттерна выделяем свои рекомендации. Но на самом деле, все пользователи немного отличаются друг от друга даже в одном кластере, поэтому возникают неточности.

- Если в кластере никто не оценивал объект, то предсказание сделать не получится.

## USER-BASED

Для того, чтобы преодолеть эти сложности, можно обратиться к другому алгоритму, который позволяет уйти от решения задачи кластеризации — User-based.

В этом алгоритме мы заменяем жесткую кластеризацию на следующую формулу:

$$ \hat{r_{ui}} = \bar{r_u}+\frac{\sum_{v \in U_i} {sim(u, v)(r_{ui}-\bar{r_v})}}{\sum_{v \in U_i} sim(u,v)} $$

Разберемся с обозначениями, которые используются в этой формуле:

$\bar{r_u}$ — средняя оценка пользователя $u$

$\bar{r_v}$— средняя оценка пользователя $v$

Оценка пользователя $\hat{r_{ui}}$, которую мы предсказываем для него, состоит из двух частей:

- Непосредственно его средняя оценка.
- Слагаемое, состоящее из: $r_{ui} - \bar{r_v}$  — разница в оценках с другими пользователями, т. е. похожесть пользователей. Эта разница домножается на похожесть пользователей. То есть в числителе оказалась средневзвешенная разница в оценках. А в знаменателе находится сумма показателей схожести.

По каждому клиенту подбираем релевантный для него товар в рамках группы клиентов, но не решаем задачу кластеризации, а усредняем интересы данной группы в дистанции нескольких соседей.

Здесь мы руководствуемся по сути идеей, что видео можно порекомендовать человеку, если оно понравится его друзьям.

## ITEM-BASED

Однако, если мы транспонируем матрицу предпочтений и будем решать ту же самую задачу не в рамках клиентов, а в рамках items, то мы получим более устойчивое решение.

$$ \hat{r_{ui}} = \bar{r_i}+\frac{\sum_{j \in I_u} {sim(i,j)(r_{uj}-\bar{r_j})}}{\sum_{j \in I_u} sim(i,j)} $$

По формуле (которая очень похожа на формулу из предыдущего подхода) можно понять, что этот подход симметричен и использует ту же самую идею. Только теперь у нас не пользователи похожи, а объекты (items) похожи.

То есть, если мы говорим о рекомендации фильмов, то мы теперь рекомендуем пользователю фильм, который похож на те фильмы, которые уже понравились этому пользователю ранее.

Кроме того, у нас будет больше размерность на каждый вектор items, чем размерность вектора клиентов по items. За счёт этого будет больше оценок, выше статистическая значимость и модель будет более устойчива к переобучению. 

### Преимущества

- Меньше размерность матрицы расстояний
- Легче вычислять
- Модель более устойчива к переобучению
- Можно реже обновлять
- Меньше подвержены изменению предпочтений со временем

### Недостатки

- Проблема холодного старта
- Плохие предсказания для новых/нетипичных пользователей/объектов
- Тривиальность рекомендаций
- Ресурсоемкость вычислений

# 9.6. Практика

Продолжим работать с датасетом Netflix.

Возьмём подвыборку из 10000 случайных кастомеров и 5000 фильмов. 

In [11]:
import time
import pandas as pd
import numpy as np
import surprise
from surprise import Reader, Dataset
from surprise import KNNBasic

In [26]:
df = pd.read_csv('data_fin.csv', sep=';')

In [27]:
cust_sample = df.Cust_Id.sample(300)
movie_sample = df.Movie_Id.sample(10000)

Для генерации простых рекомендаций с помощью коллаборативной фильтрации можно воспользоваться модулем surprise. Загрузим в модуль surprise наш датасет с помощью метода Reader.

Возьмем только те оценки, которые относятся к выбранному подмножеству кастомеров и только те оценки, которые относятся к выбранному подмножеству фильмов. Именно в такой последовательности — сначала Cust_Id, затем Movie_Id, затем Rating.

In [28]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[df.Cust_Id.isin(cust_sample) &
                              df.Movie_Id.isin(movie_sample)][['Cust_Id', 'Movie_Id', 'Rating']], reader)

В модуле surprise есть несколько реализаций коллаборативной фильтрации. Мы возьмем одну из самых самых простых — принцип ближайших соседей.

Принцип коллаборативной фильтрации заключается в следующем:

Для каждого человека находится небольшое множество похожих на него зрителей с оценками примерно такими же, какие поставил человек на ряд фильмов (item). Из этой группы можно усреднить оценки на просмотренные фильмы и для тех членов группы, у которых ещё не было просмотров этих фильмов экстраполировать значения оценок в этих ячейках.

Таким образом, у нас появляется некая средняя оценка в группе для каждого фильма из просмотренных, и мы можем предположить, что тем людям, которые ещё не успели посмотреть эти фильмы, они понравятся.

Так как размерность по пользователям больше, чем размерность по фильмам, то выгоднее использовать не user-based алгоритм, а item-based. В этом случае вектор будет состоять не из оценок одного пользователя на различные фильмы, а будет содержать все оценки фильма от многих пользователей. Таким образом мы получим больший вектор, но само количество векторов будет меньше. А если меньше количество векторов, то проще посчитать матрицу из взаимной дистанции.

Именно это мы задаем в качестве параметров алгоритма:

In [29]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)
trainingSet = data.build_full_trainset()

Запускаем алгоритм и формирует датасет для тренировки специальной функцией build_full_trainset().

После этого проводим тренировку модели на сформированном тренировочном датасете:

In [30]:
start_time = time.time()
knn.fit(trainingSet)
print("--- %s seconds ---" % np.around(time.time() - start_time, 3))

Computing the cosine similarity matrix...
Done computing similarity matrix.
--- 18.98 seconds ---


С помощью натренированной модели мы можем проскорить остальные оценки. Для этого сгенерируем тестовый сет и построим предсказание по этому датасету:

In [31]:
start_time = time.time()

testSet = trainingSet.build_anti_testset()
predictions = knn.test(testSet)

print("--- %s seconds ---" % (time.time() - start_time))

--- 320.2757239341736 seconds ---


Результат получился не удобочитаемым. Поэтому давайте сделаем вспомогательную функцию, которая будет брать топ-3 фильмов и их оценки:

In [66]:
from collections import defaultdict
 
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [67]:
#Обрабатываем наше предсказание:
top3_recommendations = get_top3_recommendations(predictions)

С помощью следующей функции переведем тексты фильмов в удобочитаемый вид, то есть раскодируем заглавия фильмов. 

In [68]:
import numpy as np
def print_recs(i):
    for (a, b) in top3_recommendations[i]:
        print(titles[titles.Movie_Id == a]['Name'].values[0], np.round(b,2))

С помощью этой функции выведем рекомендации для случайного пользователя (предварительно загрузим movie_titles.csv):

In [69]:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

In [70]:
i = np.random.choice(list(top3_recommendations.keys()))

print_recs(i)

Stand by Me 4.43
Full Metal Jacket 4.43
Memento 4.38


Давайте посмотрим, что смотрел этот человек, и выберем из нашего датасета те фильмы, которые этот человек оценил на 5.

In [71]:
films = data.df[(data.df.Cust_Id == i) & (data.df.Rating == 5)]['Movie_Id'].values
titles[titles.Movie_Id.isin(films)]['Name'].values

array(['High Fidelity', 'American Beauty', 'Clerks',
       "Aliens: Collector's Edition", "Alien: Collector's Edition",
       'Lord of the Rings: The Fellowship of the Ring', 'Braveheart',
       'Napoleon Dynamite', 'The Godfather', 'The Last Samurai',
       'Batman Begins', 'The Sixth Sense',
       'Dances With Wolves: Special Edition', 'Heat: Special Edition',
       'True Romance', 'The Right Stuff', 'I',
       'Star Wars: Episode V: The Empire Strikes Back',
       'GoodFellas: Special Edition', 'The Sopranos: Season 3',
       'Fight Club', 'Sin City', 'Donnie Brasco', 'Good Morning',
       'Spanglish', 'Gremlins', 'Billy Madison', 'Armageddon',
       'The Usual Suspects',
       'Lord of the Rings: The Two Towers: Extended Edition',
       'The Princess Bride',
       'The Lord of the Rings: The Fellowship of the Ring: Extended Edition',
       'Apollo 13', 'Willow', 'The Fog of War', 'Happy Gilmore',
       'Good Will Hunting', 'The Goonies', 'The Natural',
       'The F

# 9.9. Заключение

# 9.10. Задачи (7 задач)